In [25]:
import random
import numpy as np
from PIL import Image

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage import data, img_as_float, exposure
from skimage.filters import unsharp_mask

from IPython.display import display
import cv2
import os

In [26]:
def extract_images(folder_path):
  images = []
  for filename in os.listdir(folder_path):
    if filename.lower().endswith((".jpg", ".png", ".jpeg")):
      image_path = os.path.join(folder_path, filename)
      if os.path.isfile(image_path):
        try:
          img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
          if img is None:
            print(f"Error loading image {filename}")
            continue
          images.append(img)
        except Exception as e:
          print(f"Error processing image {filename}: {e}")
  return images


In [27]:
#image tecnique implementation

def gamma_correction(image, gamma):
  if image is None:
    print("Image not found")
    return None
  org_imag_float = img_as_float(image)
  img_gamma = exposure.adjust_gamma(org_imag_float, gamma) # display(Image.fromarray((gamma_corrected * 255).astype(np.uint8)))
  return (img_gamma*255).astype(np.uint8)

def gaussian_blur(image, sigma):
  if image is None:
    print("Image not found")
    return None
  return cv2.GaussianBlur(image, (5,5), sigma)


def unsharp_masking(image, amount):
  if image is None:
    print("Image not found")
    return None
  org_img = img_as_float(image) #this is done unsharp_mask expects float type image
  img_unsharp = unsharp_mask(org_img, amount)
  return (img_unsharp * 255).astype(np.uint8)

def histogram_equalisation(image):
  if image is None:
    print("Image not found")
    return None
  return cv2.equalizeHist(image)


def contrast_stretching(image):
  if image is None:
    print("Image not found")
    return None
  min_val = np.min(image)
  max_val = np.max(image)

  if  max_val-min_val==0:
    return image.copy()

  con_image = ((image - min_val) / (max_val - min_val)) * 255

  return con_image.astype(np.uint8)


In [28]:
technique_dict = {
    "Gamma Correction": {"gamma": lambda: random.uniform(0.5, 2.0)},
    "Gaussian Blur": {"sigma": lambda: random.uniform(0.5, 5.0)},
    "Unsharp Masking": {"amount": lambda: random.uniform(0.5, 2.0)},
    "Histogram Equalisation": {},
    "Contrast Stretching": {},
}

In [29]:
from tabulate import tabulate
def ran_pipeline(min_len=4, max_len=5):
  len = random.randint(min_len, max_len)
  pipeline = []
  img_tech = list(technique_dict.keys()) #this will get and use the keys from the dictionary technique_dict
  for x in range(len):
    tech = random.choice(img_tech)
    params = technique_dict[tech]
    pipeline.append({"tech":tech, "params":params})
  return pipeline

In [30]:
def image_to_pipeline(pipeline, image):
  if image is None:
    print("Image not found")
    return None
  img_enhanced = image.copy()

  for step in pipeline:
    tech = step["tech"]
    if tech == "Gamma Correction":
      img_enhanced = gamma_correction(img_enhanced, step["params"]["gamma"]())
    elif tech == "Gaussian Blur":
      img_enhanced = gaussian_blur(img_enhanced, step["params"]["sigma"]())
    elif tech == "Unsharp Masking":
      img_enhanced = unsharp_masking(img_enhanced, step["params"]["amount"]())
    elif tech == "Histogram Equalisation":
      img_enhanced = histogram_equalisation(img_enhanced)
    elif tech == "Contrast Stretching":
      img_enhanced = contrast_stretching(img_enhanced)
  return np.clip(img_enhanced,0,255).astype(np.uint8)



In [31]:
#fitness Function
def fitness_function(population, train_output,train_input):
  fit_scores = []

  if len(train_input)!= len(train_output):
    raise ValueError("Number of training input images must match training output images.")
  if not train_output:
    print("There are no training images provided")
    return []

  for pipeline in population:
    pipeline_final_score = 0.0
    for x in range(len(train_output)):
      output_image = train_output[x]
      input_image = train_input[x]
      image_enhance = image_to_pipeline(pipeline, input_image)
      current_score = psnr(output_image, image_enhance,data_range=255) + ssim(output_image, image_enhance,data_range=255)
      pipeline_final_score+=current_score

    avg_fit = pipeline_final_score / len(train_output)
    fit_scores.append(avg_fit)
  return fit_scores


In [32]:
#Population Random
def random_population(pop_size):
  population = []
  for x in range(pop_size):
    population.append(ran_pipeline())
  return population



In [33]:
#selection --> tournment selction will be used
def select_parents(population,fitnesses, k=3):
  select_individuals = random.sample(range(len(population)), k)
  best_individuals = max(select_individuals, key=lambda x: fitnesses[x])
  return population[best_individuals]


In [34]:
#crossover
def crossover(parent1, parent2,cross_rate):
  if len(parent1) < 2 or len(parent2) < 2:
    return parent1.copy(), parent2.copy()

    section_1 = random.randint(0, len(parent1) - 1)
    section_2 = random.randint(0, len(parent2) - 1)

    offspring1 = parent1[:section_1] + parent2[section_1:]
    offspring2 = parent2[:section_2] + parent1[section_2:]

    return offspring1, offspring2

In [35]:
#mutation
def mutation(offspring, mutate_rate):
  pass

In [36]:
#GA LOOP
def run_GA(train_input,train_output,pop_size, gens, cross_rate, mutate_rate):
  population = random_population(pop_size)
  pop_fit = fitness_function(population, train_output,train_input)
  for x in range(gens):
    offspring = []

    for y in range(pop_size//2):
      parent1, parent2 = select_parents()
      offspring1, offspring2 = crossover(parent1, parent2,cross_rate)
      offspring1 = mutation(offspring1,mutate_rate)
      offspring2 = mutation(offspring2,mutate_rate)
      offspring.append(offspring1)
      offspring.append(offspring2)


  pass


In [37]:
#VNS

In [38]:
#main
if __name__ == "__main__":
  seed_value = 42
  pop_size = 30
  gens = 100
  cross_rate = 0.8
  mutate_rate = 0.2

  train_input_pics = extract_images("/content/drive/MyDrive/COS791 ASSIGNMENT ONE DATA/Test_data")
  train_output_pics = extract_images("/content/drive/MyDrive/COS791 ASSIGNMENT ONE DATA/Training_data")

  print("Beginning with GA optimization")
  best_ga_pipeline, best_ga_fitness = run_GA(train_input_pics,train_output_pics,pop_size,gens,cross_rate,mutate_rate)
  print("Genetic Algorithm optimization completed.")
  print(f"Best GA pipeline found: {best_ga_pipeline}")
  print(f"Best GA fitness score: {best_ga_fitness}")

  print("\nBeginning with VNS optimization")
  #best_vns_pipeline, best_vns_fitness = run_VNS(train_input_pics,train_output_pics,pop_size,gens,cross_rate,mutate_rate)


Beginning with GA optimization
There are no training images provided


TypeError: select_parents() missing 2 required positional arguments: 'population' and 'fitnesses'